In [1]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from pathlib import Path
import scipy.sparse as sparse
import numpy as np
import random

In [2]:
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import SVD
from surprise import SVDpp
MY_SEED = 42
random.seed(MY_SEED)
np.random.seed(MY_SEED)

In [3]:
df_sub = pd.read_csv(Path.cwd() / 'data' / 'sample_submission.csv')
df_train = pd.read_csv(Path.cwd() / 'data' / 'train.csv')
df_test = pd.read_csv(Path.cwd() / 'data' / 'test.csv')

/home/alex/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/alex/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df_train.drop_duplicates(inplace=True)

In [5]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(0,1))
data = Dataset.load_from_df(df_train[['userid', 'itemid', 'rating']], reader)

In [6]:
trainset = data.build_full_trainset()

In [9]:
algo = SVD()

In [8]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.3426  0.3416  0.3415  0.3401  0.3417  0.3415  0.0008  
MAE (testset)     0.2303  0.2293  0.2291  0.2284  0.2290  0.2292  0.0006  
Fit time          116.55  118.67  120.99  114.75  118.94  117.98  2.14    
Test time         4.79    4.75    4.59    4.68    5.21    4.81    0.21    


{'test_rmse': array([0.34257206, 0.34159957, 0.34150402, 0.34005636, 0.34165962]),
 'test_mae': array([0.23026325, 0.22931839, 0.22907276, 0.22838353, 0.22904739]),
 'fit_time': (116.54742646217346,
  118.6669671535492,
  120.98648238182068,
  114.75357866287231,
  118.93942594528198),
 'test_time': (4.794326066970825,
  4.749876976013184,
  4.592046737670898,
  4.684158563613892,
  5.20811653137207)}

In [10]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.3484  0.3476  0.3479  0.3488  0.3478  0.3481  0.0004  
MAE (testset)     0.2362  0.2358  0.2364  0.2363  0.2360  0.2361  0.0002  
Fit time          33.66   33.74   33.97   33.90   33.81   33.82   0.11    
Test time         1.82    1.69    2.21    1.74    1.71    1.83    0.19    


{'test_rmse': array([0.3484415 , 0.34763533, 0.34787727, 0.34882031, 0.34782557]),
 'test_mae': array([0.23620133, 0.23584893, 0.23640312, 0.23625295, 0.23603404]),
 'fit_time': (33.66257882118225,
  33.74075627326965,
  33.969441175460815,
  33.902092933654785,
  33.805949449539185),
 'test_time': (1.8189184665679932,
  1.6899518966674805,
  2.211038589477539,
  1.736081600189209,
  1.7058475017547607)}

In [20]:
from surprise import CoClustering
algo = CoClustering()

In [21]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm CoClustering on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.3729  0.3801  0.3836  0.3830  0.3718  0.3783  0.0050  
MAE (testset)     0.2249  0.2210  0.2211  0.2291  0.2247  0.2241  0.0030  
Fit time          29.77   29.28   29.84   29.67   30.22   29.76   0.30    
Test time         1.64    1.57    1.57    1.58    1.58    1.59    0.03    


{'test_rmse': array([0.37293395, 0.38009022, 0.38355408, 0.38301337, 0.37180911]),
 'test_mae': array([0.22487822, 0.22095595, 0.22109129, 0.22913082, 0.22469053]),
 'fit_time': (29.76620388031006,
  29.280840396881104,
  29.83963966369629,
  29.672457456588745,
  30.222164630889893),
 'test_time': (1.6401867866516113,
  1.573112964630127,
  1.5734689235687256,
  1.5823619365692139,
  1.5759215354919434)}

In [22]:
algo.fit(trainset)

In [23]:
df_sub['rating'] = df_test.apply(lambda x: algo.predict(x.userid, x.itemid)[3], axis=1)

In [24]:
df_sub.to_csv('submission_surprise_log.csv', index=False)